In [1]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing 
from IPython.display import clear_output, Image, display, HTML
from math import floor

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)

###

split_index=floor(housing.data.shape[0] * 0.9)
data_train=housing.data[:split_index]
data_test=housing.data[split_index:]
target_train=housing.target[:split_index]
target_test=housing.target[split_index:]

'''
e.g.
points:
(1, 3), (2, 5), (3, 4)

y = ax + b, find a & b to get MSE(Min Square Error)
3 = 1a + b
5 = 2a + b
4 = 3a + b

represented as matrices:
| 1 1 |         | 3 |
| 2 1 | | a | = | 5 |
| 3 1 | | b |   | 4 |
   X    theta =   y

find theta to get MSE:
theta = (XT * X)^-1 * XT * y
'''
# y = ax+b
# 
# X * theta = y
# theta = coefficients

# X = [1s,data]
housing_data_plus_bias = np.c_[np.ones((data_train.shape[0], 1)), data_train]

# theta = (XT * X)^-1 * XT * y
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(target_train.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
# original theta
# theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

# regularized linear regression
# theta = (XT * X + lambda * I)^-1 * XT * y
lamb = 0.8
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.add(tf.matmul(XT, X), tf.scalar_mul(lamb, tf.eye(int(X.shape[1]))))), XT), y)
# X_test = [1s,data]
X_test = tf.constant(np.c_[np.ones((data_test.shape[0], 1)), data_test], dtype=tf.float32, name="X_test")

# Use theta of X to predict the y of X_test
y_pred = tf.matmul(X_test, theta, name="predictions")

# Real y of X_test
y_test = tf.constant(target_test.reshape(-1, 1), dtype=tf.float32, name="y_test")

# calculate the error rate
error = tf.reduce_mean(abs(y_pred - y_test)/y_test)
###

###### Implement Data Preprocess here ######

###### Start TF session ######
with tf.Session() as sess:
    
###
    print(error.eval())
    
    ''' 
    explain the graph :
    
    transpose of matrix X , multiply X ==> XT*X
    ↓
    eye of matrix X multiply scalar ==> I * lambda
    ↓
    add above two togather,then inverse matrix ==> (XT * X + lambda * I)^-1
    ↓
    multiply XT ==>  (XT * X + lambda * I)^-1*XT
    ↓
    multiply y ==>  (XT * X + lambda * I)^-1*XT*y
    ↓
    use theta to get the predictions with X_test.
    ↓
    prediction subtract y_test , then abs ,then divided by y_test ==> abs(y_pred - y_test)/y_test
    ↓
    compute the mean => tf.reduce_mean(abs(y_pred - y_test)/y_test)
    
    
    '''

###

    show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######

c:\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Shape of dataset: (20640, 8)
Shape of label: (20640,)
Tensor("MatMul_2:0", shape=(9, 1), dtype=float32)
0.334137
